# Reading ModelSEED Biochemistry from ModelSEED Database

This notebook provides examples to read data from the biochemistry database

The biochemistry database is represented by the `modelseedpy.biochem.modelseed_biochem.ModelSEEDDatabase` class

The database can be instantiated from either a local github copy of the database repository or loaded via web url from github.
* `from_local` - from_local(*path to repository*)
    * Example:
        1. first we obtain a copy of the repository: `git clone https://github.com/ModelSEED/ModelSEEDDatabase.git`
        2. then we can load the repository with `from_local`: `from_local('/home/user/ModelSEEDDatabase')`
* `from_github` - from_github(*commit version*)
    * Example:
        * load the dev branch from github (default: https://github.com/ModelSEED/ModelSEEDDatabase): `from_github('dev')`
        * load a specific commit version from github: `from_github('194ac8afe48f8a606c0dd07ba3c7af10c02ba2fd')`
        * load from another fork/repository: `from_github('master', 'https://raw.githubusercontent.com/Fxe/ModelSEEDDatabase')`

In [1]:
import logging
import pandas as pd
from modelseedpy.biochem.modelseed_biochem import get_structures_from_df, get_aliases_from_df, get_names_from_df, process_aliases, load_metabolites_from_df
from modelseedpy.biochem.modelseed_biochem import ALIAS_RXN_IDENTIFIERS_ORG, ALIAS_MODELS
from modelseedpy.biochem.modelseed_biochem import from_local2, from_local, from_github
from modelseedpy.biochem.modelseed_reaction import ModelSEEDReaction, ModelSEEDReaction2
logger = logging.getLogger(__name__)

In [2]:
%%time
modelseed_local2 = from_local2('/home/fliu/workspace/python/ModelSEEDDatabase/')

CPU times: user 16.8 s, sys: 668 ms, total: 17.5 s
Wall time: 17.5 s


In [3]:
%%time
modelseed_local1 = from_local('/home/fliu/workspace/python/ModelSEEDDatabase/')

CPU times: user 42.2 s, sys: 503 ms, total: 42.7 s
Wall time: 42.7 s


In [4]:
%%time
modelseed_git = from_github('dev')

CPU times: user 18.8 s, sys: 1.12 s, total: 19.9 s
Wall time: 24.3 s


In [5]:
inchi_key_lookup = {}
for cpd in modelseed_git.compounds:
    inchi_key = cpd.inchi_key
    inchi_key = None if pd.isna(inchi_key) or len(inchi_key) == 0 else inchi_key
    cpd.inchi_key = inchi_key
    if cpd.inchi_key:
        a, b, p = cpd.inchi_key.split('-')
        if a not in inchi_key_lookup:
            inchi_key_lookup[a] = {}
        if b not in inchi_key_lookup[a]:
            inchi_key_lookup[a][b] = set()
        inchi_key_lookup[a][b].add((cpd.id, p))

In [6]:
metabolite_reactions = {}
for rxn in modelseed_git.reactions:
    if not rxn.is_obsolete:
        for m in rxn.metabolites:
            if m.seed_id not in metabolite_reactions:
                metabolite_reactions[m.seed_id] = set()
            metabolite_reactions[m.seed_id].add(rxn.id)

In [14]:
import cobrakbase
kbase = cobrakbase.KBaseAPI()
genome = kbase.get_from_ws('87388/3/1')

In [38]:
genome.features.get_by_id('A3776_RS08395').data

{'aliases': [['protein_id', 'WP_066378293.1'], ['locus_tag', 'A3776_RS08395']],
 'cdss': ['A3776_RS08395_CDS_1'],
 'dna_sequence': 'ATGATTGTACCCTTGCTATATCTGGCTTTAGCCGGAGCCTATCTATTAGTTGTCCCGGTGGCTTTAATGCTTTACCTCAACCTACGCTGGTATACGGCTGGCTCGATTGAGCGCACCGTCATGTATTTTTTTGTATTTTTGTTCTTTCCCGGACTGTTGGTTTTGTCGCCGTTTGTGAATCTGCGACCCAAACCCCGCAAAATTGAAGTTTAA',
 'dna_sequence_length': 213,
 'functions': ['NADH dehydrogenase subunit', 'NdhL'],
 'id': 'A3776_RS08395',
 'location': [['NZ_LUHI01000036.1', 21434, '-', 213]],
 'md5': '1ae8b7b3d705b63184c99a9711785d23',
 'protein_md5': '1ae8b7b3d705b63184c99a9711785d23',
 'protein_translation': 'MIVPLLYLALAGAYLLVVPVALMLYLNLRWYTAGSIERTVMYFFVFLFFPGLLVLSPFVNLRPKPRKIEV',
 'protein_translation_length': 70,
 'quality': {'hit_count': 5, 'weighted_hit_count': 5.5205}}

In [221]:
import json
data = None
with open('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single.json', 'r') as fh:
    data = json.load(fh)

In [222]:
%run /home/fliu/workspace/data/sbml/iAnC892/tools.py
integrate_to_seed(data)

In [161]:
with open('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single_mapped.json', 'w') as fh:
    fh.write(json.dumps(data, indent=4, sort_keys=True))

In [224]:
model_mapped.metabolites.cpd00011_p

Metabolite identifier,cpd00011_p
Name,CO2
Memory address,0x07fcba66f68e0
Formula,CO2
Compartment,p
In 2 reaction(s),"cpd00011exccc_p, cpd00011excpp_e"


In [225]:
model_mapped = cobra.io.load_json_model('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single_mapped.json')
from modelseedpy import MSBuilder
from cobra.core import Reaction
MSBuilder.build_exchanges(model_mapped, 'e')
atp_obj = Reaction('obj_ATP', 'ATP objective function', 'objetive function', 0, 1000)
atp_obj.add_metabolites({
    model_mapped.metabolites.cpd00001_c: -1,
    model_mapped.metabolites.cpd00002_c: -1,
    model_mapped.metabolites.cpd00008_c: 1,
    model_mapped.metabolites.cpd00009_c: 1,
    model_mapped.metabolites.cpd00067_c: 1,
})
model_mapped.add_reactions([atp_obj])
model_mapped.objective = 'obj_ATP'
model_mapped.medium = {
    #'EX_cpd00067_e': 1,
    'EX_cpd11632_e': 0.5,
    #'EX_hvphoton2_e': 0.5,
    'EX_hvphoton1_e': 0.5
}
#1.0 obj_ATP = 0.8035714285714285
model_mapped.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd11632_e,EX_cpd11632_e,0.5,0,0.00%
hvphoton1_e,EX_hvphoton1_e,0.5,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [197]:
model_mapped.medium = {
        'EX_cpd00067_e': 1
    }
sol = cobra.flux_analysis.pfba(model_mapped)

In [ ]:
for a in [{'EX_cpd11632_e': 1}, {'EX_hvphoton1_e': 1}]:
    model_mapped.medium = {
        k.id: 1,
    }
    sol = cobra.flux_analysis.pfba(model_mapped)

    print(model_mapped.medium)
    ss = f"{cpd.id}\t{cpd.name}\t{sol.fluxes['obj_ATP']}\t{sol.status}"
    print(ss)

In [ ]:
#glc/O2 22.857142857142854
#glc 0

In [218]:
model_mapped.medium = {
    'EX_cpd00067_e': 1000,
    'EX_cpd00001_e': 1000,
    'EX_cpd00242_e': 1, #H2CO3
    'EX_cpd00528_e': 1000, #N2
    'EX_cpd00011_e': 1000, #co2
    'EX_cpd00007_e': 0, #O2
    'EX_cpd11632_e': 0,
    'EX_hvphoton1_e': 0,
    'EX_cpd00027_e': 1,
    'EX_cpd00048_e': 1,
    'EX_cpd00209_e': 1,
    'EX_cpd11640_e': 1, #H2
    'EX_cpd00268_e': 10
    }
model_mapped.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [203]:
for k in model_mapped.exchanges:
    cpd = list(k.metabolites)[0]
    model_mapped.medium = {
        k.id: 1,
    }
    sol = cobra.flux_analysis.pfba(model_mapped)
    ss = f"{cpd.id}\t{cpd.name}\t{sol.fluxes['obj_ATP']}\t{sol.status}"
    print(ss)

cpd11632_e	Photosystem 2 photon	0.8571428571428571	optimal
hvphoton1_e	Photosystem 1 photon	0.75	optimal
cpd00067_e	H+	0.0	optimal
cpd00001_e	H2O	0.0	optimal
cpd00007_e	O2	0.0	optimal
cpd00009_e	Phosphate	0.0	optimal
cpd00048_e	Sulfate	0.0	optimal
cpd00209_e	Nitrate	0.0	optimal
cpd00254_e	Mg	0.0	optimal
cpd10515_e	Fe+2	0.0	optimal
cpd00149_e	Co2+	0.0	optimal
cpd00034_e	Zn2+	0.0	optimal
cpd00058_e	Cu2+	0.0	optimal
cpd20863_e	Mn2+	0.0	optimal
cpd00063_e	Ca2+	0.0	optimal
cpd00205_e	K+	0.0	optimal
cpd00971_e	Na+	0.0	optimal
cpd11574_e	Molybdate	0.0	optimal
cpd00011_e	CO2	0.0	optimal
cpd00528_e	N2	0.0	optimal
cpd00027_e	D-Glucose	0.0	optimal
cpd00053_e	L-Glutamine	0.0	optimal
cpd00107_e	L-Leucine	0.0	optimal
cpd00118_e	Putrescine	0.0	optimal
cpd00129_e	L-Proline	0.0	optimal
cpd00156_e	L-Valine	0.0	optimal
cpd00179_e	Maltose	0.0	optimal
cpd00264_e	Spermidine	0.0	optimal
cpd00268_e	H2S2O3	0.0	optimal
cpd00322_e	L-Isoleucine	0.0	optimal
cpd03725_e	Fe(III)dicitrate	0.0	optimal
cpd00060_e	L-Meth

In [43]:
for k in ['A3776_RS12690', 'A3776_RS16110']:
    for o in genome.features.get_by_id(k).functions:
        print(o)


Dihydroorotate dehydrogenase (quinone) (EC 1.3.5.2)
Putative dihydropyrimidine dehydrogenase [NADP+], similar to dihydroorotate dehydrogenase


In [19]:
for rxn_id in super_c:
    if rxn_id in model.reactions:
        stoich_key, compartment_setup = super_c[rxn_id]
        rxn_model = model.reactions.get_by_id(rxn_id)
        print(rxn_model.id, rxn_model.gene_reaction_rule)

NDH1_1u_k A3776_RS06230 and A3776_RS16180 and A3776_RS20905 and A3776_RS06215 and A3776_RS06220 and A3776_RS19180 and ( A3776_RS06225 or A3776_RS21975 ) and A3776_RS20895 and A3776_RS20900 and A3776_RS08395 and A3776_RS11390 and A3776_RS03245 and A3776_RS09810 and A3776_RS09805 and A3776_RS09800 and ( A3776_RS08060 or A3776_RS10415 or A3776_RS22280 ) and ( A3776_RS03760 or A3776_RS08065 or A3776_RS18720 )
NDH1_1p_p A3776_RS06230 and A3776_RS16180 and A3776_RS20905 and A3776_RS06215 and A3776_RS06220 and A3776_RS19180 and ( A3776_RS06225 or A3776_RS21975 ) and A3776_RS20895 and A3776_RS20900 and A3776_RS08395 and A3776_RS11390 and A3776_RS03245 and A3776_RS09810 and A3776_RS09805 and A3776_RS09800 and ( A3776_RS08060 or A3776_RS10415 or A3776_RS22280 ) and ( A3776_RS03760 or A3776_RS08065 or A3776_RS18720 )
NDH1_4pp_p A3776_RS06230 and A3776_RS16180 and A3776_RS20905 and A3776_RS06215 and A3776_RS06220 and A3776_RS19180 and ( A3776_RS06225 or A3776_RS21975 ) and A3776_RS20895 and A3776_

In [7]:
import cobra
model = cobra.io.load_json_model('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_single.json')

In [64]:
model.reactions.CBFCu_k

Reaction identifier,CBFCu_k
Name,cytochrome b6/f complex
Memory address,0x07fcbbf9f9f70
Stoichiometry,2.0 cpd00067_c + 2.0 cpd12265_k + cpd16503_k --> 4.0 cpd00067_k + cpd07274_k + 2.0 cpd12239_k 2.0 H+ + 2.0 Oxidized plastocyanin + Plastoquinol-9 --> 4.0 H+ + Plastoquinone A + 2.0 Reduced plastocyanin
GPR,A3776_RS13955 or A3776_RS22750
Lower bound,0.0
Upper bound,1000.0


In [62]:
seed_build = {
    'PSII_k': ('rxn46370', {'0': 'k', '1': 'c'}),
}
modelseed_git.reactions.rxn46370.to_template_reaction({'0': 'k', '1': 'c'})

cpd00007_0 0 cpd00007_k
-1.0
cpd00067_0 0 cpd00067_k
-4.0
cpd16503_0 0 cpd16503_k
-2.0
cpd00001_0 0 cpd00001_k
2.0
cpd00067_1 1 cpd00067_c
4.0
cpd07274_0 0 cpd07274_k
2.0
cpd11632_0 0 cpd11632_k
1.0


Reaction identifier,rxn46370_k
Name,nan
Memory address,0x07fcbbc01f790
Stoichiometry,cpd00007_k + 4.0 cpd00067_k + 2.0 cpd16503_k <=> 2.0 cpd00001_k + 4.0 cpd00067_c + 2.0 cpd07274_k + cpd11632_k O2 + 4.0 H+ + 2.0 Plastoquinol-9 <=> 2.0 H2O + 4.0 H+ + 2.0 Plastoquinone A + hn
GPR,
Lower bound,-1000
Upper bound,1000


In [170]:
#rxn37623: cpd00005_0 + 3.0 cpd00067_0 + cpd07274_0 <=> cpd00006_0 + cpd29046_0
#rxn37623 NADPH + 3.0 H+ + Plastoquinone A <=> NADP + plastoquinol A
#rxn36824 NADPH + H+ + Plastoquinone A <=> NADP + Plastoquinol-9
#rxn36824: cpd00005_0 + cpd00067_0 + cpd07274_0 <=> cpd00006_0 + cpd16503_0
super_c = {
    'NDH1_1u_k': ('NDH1_1u_k', {'0': 'c', '1': 'k'}),
    'NDH1_1p_p': ('NDH1_1u_k', {'0': 'c', '1': 'p'}),
    'NDH1_4pp_p': ('NDH1_3u_k', {'0': 'c', '1': 'p'}),
    'NDH1_3u_k': ('NDH1_3u_k', {'0': 'c', '1': 'k'}),
    'NDH1_2p_p': ('NDH1_2u_k', {'0': 'c', '1': 'p'}),
    'NDH1_2u_k': ('NDH1_2u_k', {'0': 'c', '1': 'k'}),
    
    'rxn08335_p_p': ('DHORD_PQ9_k', {'0': 'c', '1': 'p'}),
    'DHORD_PQ9_k': ('DHORD_PQ9_k', {'0': 'c', '1': 'k'}),
    'FQR_k': ('FQR_k', {'0': 'c', '1': 'k'}),
    'CBFCu_k': ('CBFCu_k', {'0': 'c', '1': 'k'}),
}
#           cpd00005_c + 4.0 cpd00067_c + cpd07274_k --> cpd00006_c + 3.0 cpd00067_k + cpd16503_k
#NDH1_1p_p: cpd00005_c + 4.0 cpd00067_c + cpd07274_p --> cpd00006_c + 3.0 cpd00067_p + cpd16503_p
#NDH1_1u_k: cpd00005_c + 4.0 cpd00067_c + cpd07274_k --> cpd00006_c + 3.0 cpd00067_k + cpd16503_k
super_s = {
    'NDH1_1u_k': {
        ('cpd00005', '0'): -1, 
        ('cpd00067', '0'): -4, 
        ('cpd07274', '1'): -1,
        ('cpd00006', '0'): 1, 
        ('cpd00067', '1'): 3, 
        ('cpd16503', '1'): 1
    },

#NDH1_4pp_p: cpd00001_c + cpd00005_c + cpd00011_p + 3.0 cpd00067_c + cpd07274_p --> cpd00006_c + 3.0 cpd00067_p + cpd00242_c + cpd16503_p
#NDH1_3u_k : cpd00001_c + cpd00005_c + cpd00011_p + 3.0 cpd00067_c + cpd07274_k --> cpd00006_c + 3.0 cpd00067_k + cpd00242_c + cpd16503_k
    'NDH1_3u_k': {
        ('cpd00001', '0'): -1, 
        ('cpd00005', '0'): -1, 
        ('cpd00011', '0'): -1,
        ('cpd00067', '0'): -3,
        ('cpd07274', '1'): -1,
        ('cpd00006', '0'): 1, 
        ('cpd00067', '1'): 3, 
        ('cpd00242', '0'): 1,
        ('cpd16503', '1'): 1
    },
#NDH1_2p_p: cpd00004_c + 4.0 cpd00067_c + cpd07274_p --> cpd00003_c + 3.0 cpd00067_p + cpd16503_p
#NDH1_2u_k: cpd00004_c + 4.0 cpd00067_c + cpd07274_k --> cpd00003_c + 3.0 cpd00067_k + cpd16503_k
    'NDH1_2u_k': {
        ('cpd00004', '0'): -1, 
        ('cpd00067', '0'): -4, 
        ('cpd07274', '1'): -1,
        ('cpd00003', '0'): 1,
        ('cpd00067', '1'): 3, 
        ('cpd16503', '1'): 1
    },
#NDH2_syn_k: cpd00004_c + cpd00067_c + cpd07274_k --> cpd00003_c + cpd16503_k
    'NDH2_syn_k': {
        ('cpd00004', 'c'): -1, 
        ('cpd00067', 'c'): -1, 
        ('cpd07274', 'k'): -1,
        ('cpd00003', 'c'): 1,
        ('cpd16503', 'k'): 1
    },
#rxn08335_p_p: cpd00282_c + cpd07274_p --> cpd00247_c + cpd16503_p
#DHORD_PQ9_k : cpd00282_c + cpd07274_k --> cpd00247_c + cpd16503_k
    'DHORD_PQ9_k': {
        ('cpd00282', '0'): -1, 
        ('cpd07274', '1'): -1, 
        ('cpd00247', '0'): 1,
        ('cpd16503', '1'): 1
    },
#FQR_k: 2.0 cpd00067_c + cpd07274_k + 2.0 cpd11620_c --> 2.0 cpd11621_c + cpd16503_k
    'FQR_k': {
        ('cpd00067', '0'): -2, 
        ('cpd07274', '1'): -1, 
        ('cpd11620', '0'): -2,
        ('cpd11621', '0'): 2,
        ('cpd16503', '1'): 1
    },
#CBFCu_k: 2.0 cpd00067_c + 2.0 cpd12265_k + cpd16503_k --> 4.0 cpd00067_k + cpd07274_k + 2.0 cpd12239_k
    'CBFCu_k': {
        ('cpd00067', '0'): -2, 
        ('cpd12265', '1'): -2, 
        ('cpd16503', '1'): -1,
        ('cpd00067', '1'): 4,
        ('cpd07274', '1'): 1,
        ('cpd12239', '1'): 2
    },
}

In [150]:
# CYTBDu_k -> DB with pq9

In [174]:
def test(model_mapped):
    for rxn_id in super_c:
        if rxn_id in model_mapped.reactions:
            stoich_key, compartment_setup = super_c[rxn_id]
            rxn_model = model_mapped.reactions.get_by_id(rxn_id)
            #print(rxn_id, rxn_model)
            #
            rxn_s_model = dict(map(lambda x: (x[0].id, x[1]), rxn_model.metabolites.items()))
            rxn_s_match = {}
            for (cpd_id, cmp_token), v in super_s[stoich_key].items():
                _cpd_id = f'{cpd_id}_{compartment_setup[cmp_token]}'
                rxn_s_match[_cpd_id] = v

            valid = rxn_s_model == rxn_s_match
            print(rxn_id, valid)
            if not valid:
                print(rxn_id, compartment_setup)
                print('rxn_s_match', rxn_s_match)
                print('rxn_s_model', rxn_s_model)
test(model_mapped)

NDH1_1u_k True
NDH1_1p_p True
NDH1_4pp_p True
NDH1_3u_k True
NDH1_2p_p True
NDH1_2u_k True
rxn08335_p_p True
DHORD_PQ9_k True
FQR_k True
CBFCu_k True


In [ ]:
rxn14159: cpd00006_0 + cpd00067_0 + 2.0 cpd11620_0 <=> cpd00005_0 + 2.0 cpd11621_0
rxn14159 NADP + H+ + 2.0 Reducedferredoxin <=> NADPH + 2.0 Oxidizedferredoxin

rxn05937: cpd00006_0 + cpd00067_0 + cpd11620_0 <=> cpd00005_0 + cpd11621_0
rxn05937 NADP + H+ + Reducedferredoxin <=> NADPH + Oxidizedferredoxin

In [228]:
def finddd(a, b):
    c = set(a) | set(b)
    super_match = None
    for i in c:
        if i in metabolite_reactions:
            if super_match is None:
                super_match = set(metabolite_reactions[i])
            super_match &= set(metabolite_reactions[i])
    return super_match

#finddd(['cpd00003'], ['cpd00004'])
#cpd00001_k + 2.0 cpd00067_c + cpd07274_k + 2.0 hvphoton2_c --> 0.5 cpd00007_k + 2.0 cpd00067_k + cpd16503_k
#2 cpd00001_k + 4.0 cpd00067_c + 2 cpd07274_k + 4.0 hvphoton2_c --> 1 cpd00007_k + 4.0 cpd00067_k + 2 cpd16503_k
for r in finddd(['cpd00006', 'cpd11620'], ['cpd00005', 'cpd11621']):
    rxn = modelseed_git.reactions.get_by_id(r)
    print(rxn, rxn.compartments)
    print(rxn.id, rxn.build_reaction_string(True))
    print()
    break

rxn34892: 4.0 cpd00005_0 + cpd00007_0 + cpd11621_0 + cpd30237_0 <=> 2.0 cpd00001_0 + 4.0 cpd00006_0 + cpd11620_0 + cpd30238_0 {'0'}
rxn34892 4.0 NADPH + O2 + Oxidizedferredoxin + Digalactosyldiacylglycerol (1-(9Z)-octadecenoyl,2-(7Z,10Z)-hexadecadienoyl, 18:1(9Z)/16:2(7Z,10Z)) <=> 2.0 H2O + 4.0 NADP + Reducedferredoxin + Digalactosyldiacylglycerol (1-(9Z)-octadecenoyl,2-(4Z,7Z,10Z)-hexadecatrienoyl, 18:1(9Z)/16:3(4Z,7Z,10Z))



In [ ]:
#2 cpd00001_0 + 4.0 cpd00067_1 + 2 cpd07274_0 + cpd11632_0      --> 1 cpd00007_0 + 4.0 cpd00067_0 + 2 cpd16503_0
#2 cpd00001_k + 4.0 cpd00067_c + 2 cpd07274_k + 4.0 hvphoton2_c --> 1 cpd00007_k + 4.0 cpd00067_k + 2 cpd16503_k

In [48]:
modelseed_git.reactions.rxn05937

Reaction identifier,rxn05937
Name,Ferredoxin:NADP+ oxidoreductase
Memory address,0x07fcbbd112c70
Stoichiometry,cpd00006_0 + cpd00067_0 + cpd11620_0 <=> cpd00005_0 + cpd11621_0 NADP + H+ + Reducedferredoxin <=> NADPH + Oxidizedferredoxin
GPR,
Lower bound,-1000
Upper bound,1000


In [283]:
model_full.reactions.EX_cpd00027[e]

Reaction identifier,ATP_syn_l__hc
Name,ATP synthetase(u)
Memory address,0x07fcb9a4b07f0
Stoichiometry,3.0 cpd00008[hc] + 3.0 cpd00009[hc] + 14.0 cpd00067[hl] --> 3.0 cpd00001[hc] + 3.0 cpd00002[hc] + 11.0 cpd00067[hc] 3.0 ADP + 3.0 Phosphate + 14.0 H+ --> 3.0 H2O + 3.0 ATP + 11.0 H+
GPR,( A3776_RS10860 or A3776_RS13440 ) and ( A3776_RS10820 or A3776_RS10980 ) and ( A3776_RS10975 or...
Lower bound,0.0
Upper bound,1000.0


In [759]:
model_full_mapped = cobra.io.load_json_model('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_mapped.json')
from cobra.core import Reaction
ex_hn = Reaction('EX_cpd11632_e0', 'Exchange for photon', '', -1000, 1000)
ex_hn.add_metabolites({
    model_full_mapped.metabolites.cpd11632_e0: -1
})
model_full_mapped.add_reactions([ex_hn])
model_full_mapped

Name,M_iAnC892
Memory address,0x07fcb9861eee0
Number of metabolites,1811
Number of reactions,1848
Number of groups,0
Objective expression,1.0*biomass_eq_33047_c0 - 1.0*biomass_eq_33047_c0_reverse_4b9c5
Compartments,"Vegetative cytoplasm, Vegetative lumen, Vegetative periplasm, Extracellular space, Vegetative carboxysome, Heterocyst cytoplasm, Heterocyst lumen, Heterocyst periplasm, Pseudo compartment"


In [769]:
for m in model_full.metabolites:
    if m.compartment == 'vp':
        print(m, m.name)
        for r in m.reactions:
            print('\t', r)
        print()
        print()

cpd00001[vp] H2O
	 CYO1b2pp_syn__vc: 0.5 cpd00007[vp] + 4.0 cpd00067[vc] + 2.0 cpd12239[vp] --> cpd00001[vp] + 2.0 cpd00067[vp] + 2.0 cpd12265[vp]
	 CYTBDpp__vc: 0.5 cpd00007[vp] + cpd16503[vp] --> cpd00001[vp] + cpd07274[vp]
	 rxn08687__vc: cpd00001[vp] <=> cpd00001[vc]
	 CYO1bpp_syn__vc: 0.5 cpd00007[vp] + 4.0 cpd00067[vc] + 2.0 focytc_6[vp] --> cpd00001[vp] + 2.0 cpd00067[vp] + 2.0 ficytc_6[vp]
	 rxn08686__vc: cpd00001[e] <=> cpd00001[vp]


cpd00007[vp] O2
	 CYO1b2pp_syn__vc: 0.5 cpd00007[vp] + 4.0 cpd00067[vc] + 2.0 cpd12239[vp] --> cpd00001[vp] + 2.0 cpd00067[vp] + 2.0 cpd12265[vp]
	 CYTBDpp__vc: 0.5 cpd00007[vp] + cpd16503[vp] --> cpd00001[vp] + cpd07274[vp]
	 rxn09031__vc: cpd00007[e] <=> cpd00007[vp]
	 CYO1bpp_syn__vc: 0.5 cpd00007[vp] + 4.0 cpd00067[vc] + 2.0 focytc_6[vp] --> cpd00001[vp] + 2.0 cpd00067[vp] + 2.0 ficytc_6[vp]
	 O2tpp__vc: cpd00007[vp] <=> cpd00007[vc]


cpd00067[vp] H+
	 CBFC2pp__vc: 2.0 cpd00067[vc] + cpd16503[vp] + 2.0 ficytc_6[vp] --> 4.0 cpd00067[vp] + cpd

In [633]:
#model_full_mapped.summary()
#0.18240098144906972
#cpd00242_e0	EX_cpd00242[e]	8.486
#hvphoton1_e0	EX_PHO1	61.35
#hvphoton2_e0	EX_PHO2	51.03
sol = cobra.flux_analysis.pfba(model_full_mapped)

In [629]:
sol.fluxes.to_csv('/home/fliu/workspace/data/sbml/iAnC892/solution_wt.csv')

In [733]:
model_full_json = None
with open('/home/fliu/workspace/data/sbml/iAnC892/iAnC892.json', 'r') as fh:
    model_full_json = json.load(fh)

In [734]:
%run /home/fliu/workspace/data/sbml/iAnC892/tools.py

In [735]:
model_full_json_copy = copy.deepcopy(model_full_json)

In [736]:
MyIntegrator.rename_compound('hvphoton1[vp]', 'cpd11632[e]', model_full_json_copy)
MyIntegrator.rename_compound('hvphoton1[hp]', 'cpd11632[e]', model_full_json_copy)
MyIntegrator.rename_compound('hvphoton2[vp]', 'cpd11632[e]', model_full_json_copy)
MyIntegrator.rename_compound('hvphoton1[vc]', 'cpd11632[vc]', model_full_json_copy)
MyIntegrator.rename_compound('hvphoton1[hc]', 'cpd11632[hc]', model_full_json_copy)
MyIntegrator.rename_compound('hvphoton2[vc]', 'cpd11632[vc]', model_full_json_copy)
MyIntegrator.delete_reaction('PHOtex1__vc', model_full_json_copy)
MyIntegrator.delete_reaction('PHOtex1__hc', model_full_json_copy)
MyIntegrator.delete_reaction('PHOtex2__vc', model_full_json_copy)
MyIntegrator.delete_reaction('EX_PHO2', model_full_json_copy)
MyIntegrator.delete_reaction('EX_PHO1', model_full_json_copy)
MyIntegrator.rename_compound('cpd00011[hp]', 'cpd00011[e]', model_full_json_copy)
MyIntegrator.rename_compound('cpd00011[vp]', 'cpd00011[e]', model_full_json_copy)
MyIntegrator.delete_reaction('cpd00011excpp__vc', model_full_json_copy)
MyIntegrator.delete_reaction('cpd00011excpp__hc', model_full_json_copy)
MyIntegrator.delete_reaction('EX_cpd00011[e]', model_full_json_copy)

In [771]:
remove_periplasm_rename = {
    'cpd07274[hp]': 'cpd07274[hc]',
    'cpd16503[hp]': 'cpd16503[hc]',
    'ficytc_6[hp]': 'ficytc_6[hc]',
    'focytc_6[hp]': 'focytc_6[hc]',
    'cpd12239[hp]': 'cpd12239[hc]',
    'cpd12265[hp]': 'cpd12265[hc]',
    'cpd07274[vp]': 'cpd07274[vc]',
    'cpd16503[vp]': 'cpd16503[vc]',
    'cpd12239[vp]': 'cpd12239[vc]',
    'cpd12265[vp]': 'cpd12265[vc]',
    'ficytc_6[vp]': 'ficytc_6[vc]',
    'focytc_6[vp]': 'focytc_6[vc]'
}
for i in remove_periplasm_rename:
    MyIntegrator.rename_compound(i, remove_periplasm_rename[i], model_full_json_copy)

In [737]:
dups = MyIntegrator.dups(model_full_json_copy)
print(dups.keys())
deleted = MyIntegrator.filter_dups(model_full_json_copy)
print(len(deleted))
dups = MyIntegrator.dups(model_full_json_copy)
print(dups.keys())

dict_keys(['cpd11632[vc]', 'cpd11632[e]'])
3
dict_keys([])


In [738]:
integrate = MyIntegrator(model_full_json_copy)

In [739]:
model_full_json_mapped = integrate.integrate_full_model_to_seed()

EX_PHO2
EX_PHO1
suc_vc_hc_exch
glu_vc_hc_exch
akg_vc_hc_exch
gln_vc_hc_exch
cpd11640exccc_hc
cpd11640excpp_hc
cpd11640exccc_vc
cpd11640excpp_vc


In [709]:
list(filter(lambda x: x[1] == 'hvphoton1_e0', integrate.compound_rename.items()))

[('hvphoton1[e]', 'hvphoton1_e0')]

In [740]:
from modelseedpy.core.msmodel import get_cmp_token
ids = set()
for rxn_from in integrate.reaction_rename:
    rxn_to = integrate.reaction_rename[rxn_from]
    if rxn_to in ids:
        print('!')
    ids.add(rxn_to)
    integrate.data_reactions[rxn_from]['id'] = rxn_to

In [744]:
import math
data_metabolites = dict(map(lambda x: (x['id'], x), model_full_json_mapped['metabolites']))
for r in model_full_json_mapped['reactions']:
    #print(r['id'])
    metabolites = list(map(lambda x: data_metabolites[x], r['metabolites']))
    indexes = set(map(lambda x: x['index'], metabolites))
    cmps = set(map(lambda x: x['compartment'][:-1], metabolites))
    rxn_cmp = get_cmp_token(cmps)
    if cmps == {'e', 'p'}:
        rxn_cmp = 'p'
    if cmps == {'c', 'k', 'p'}:
        rxn_cmp = 'k'
    if rxn_cmp is None:
        print(r['id'], cmps, rxn_cmp)
    elif len(indexes) == 1:
        suffix = rxn_cmp + list(indexes)[0]
        if not r['id'].endswith(suffix):
            #print(r['id'], suffix)
            pass
    flux = sol.fluxes[r['id']]
    if len(indexes) > 1 and math.fabs(flux) > 0:
        print(r['id'], cmps, indexes, flux)

PHOtpp1_c1 {'c', 'e'} {'1', '0'} 16.433701168825166
H+tex_c1 {'e', 'p'} {'0', '1'} 1.8604672397566566
rxn08686_c1 {'e', 'p'} {'0', '1'} -0.6301653188076215
rxn09031_c1 {'e', 'p'} {'0', '1'} 0.007060133469360138
cpd00011excpp_c1 {'e', 'p'} {'0', '1'} 0.9082375363436314
cpd00528excpp_c1 {'e', 'p'} {'1', '0'} -0.6772462674375839
cpd00009excpp_c1 {'e', 'p'} {'0', '1'} -0.007102530301648417
cpd00048excpp_c1 {'e', 'p'} {'0', '1'} -0.006401000613388471
cpd00254excpp_c1 {'e', 'p'} {'0', '1'} -0.0005122802570501466
cpd10515excpp_c1 {'e', 'p'} {'1', '0'} -0.0002548326163591996
cpd00149excpp_c1 {'e', 'p'} {'0', '1'} -5.7935855860126646e-05
cpd00034excpp_c1 {'e', 'p'} {'0', '1'} -5.395734017578539e-05
cpd00058excpp_c1 {'e', 'p'} {'0', '1'} -5.395734017578539e-05
cpd20863excpp_c1 {'e', 'p'} {'0', '1'} -5.395734017578539e-05
cpd00063excpp_c1 {'e', 'p'} {'0', '1'} -8.084697174853654e-05
cpd00205excpp_c1 {'e', 'p'} {'0', '1'} -0.0030344325960243667
cpd00971excpp_c1 {'e', 'p'} {'0', '1'} -6.74911944773

In [745]:
list(filter(lambda x: x[1] == 'EX_cpd0000_e0', integrate.reaction_rename.items()))

[]

In [746]:
for k in integrate.reaction_rename:
    print(k, integrate.reaction_rename[k])
    break

ATP_syn_l__vc ATP_syn_l_c0


In [760]:
model_full_mapped.reactions.cpd00011excpp_c1 

Reaction identifier,cpd00011excpp_c1
Name,cpd00011excpp
Memory address,0x07fcb94191fd0
Stoichiometry,cpd00011_p1 <=> cpd00011_e0 CO2 <=> CO2
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [747]:
with open('/home/fliu/workspace/data/sbml/iAnC892/iAnC892_mapped.json', 'w') as fh:
    fh.write(json.dumps(model_full_json_mapped))

In [343]:
model_full = cobra.io.load_json_model('/home/fliu/workspace/data/sbml/iAnC892/iAnC892.json')

In [324]:
model_full.medium = {
    'EX_PHO2': 0, #Photosystem 2 photon
'EX_PHO1': 100, #Photosystem 1 photon
'EX_cpd00067[e]': 1000, #H+
'EX_cpd00001[e]': 1000, #H2O
'EX_cpd00007[e]': 1000, #O2
'EX_cpd00009[e]': 1000, #Phosphate
'EX_cpd00048[e]': 1000, #Sulfate
'EX_cpd00209[e]': 1000, #Nitrate
'EX_cpd00254[e]': 1000, #Mg
'EX_cpd10515[e]': 1000, #Fe+2
'EX_cpd00149[e]': 1000, #Co2+
'EX_cpd00034[e]': 1000, #Zn2+
'EX_cpd00058[e]': 1000, #Cu2+
'EX_cpd20863[e]': 1000, #Mn2+
'EX_cpd00063[e]': 1000, #Ca2+
'EX_cpd00205[e]': 1000, #K+
'EX_cpd00971[e]': 1000, #Na+
'EX_cpd11574[e]': 1000, #Molybdate
'EX_cpd00011[e]': 1000, #CO2
'EX_cpd00528[e]': 0, #N2
'EX_cpd00027[e]': 1000, #D-Glucose
'EX_cpd00053[e]': 1000, #L-Glutamine
'EX_cpd00107[e]': 1000, #L-Leucine
'EX_cpd00118[e]': 1000, #Putrescine
'EX_cpd00129[e]': 1000, #L-Proline
'EX_cpd00156[e]': 1000, #L-Valine
'EX_cpd00179[e]': 1000, #Maltose
'EX_cpd00264[e]': 1000, #Spermidine
'EX_cpd00268[e]': 1000, #H2S2O3
'EX_cpd00322[e]': 1000, #L-Isoleucine
'EX_cpd03725[e]': 1000, #Fe(III)dicitrate
'EX_cpd00060[e]': 1000, #L-Methionine
'EX_cpd00104[e]': 1000, #BIOT
'EX_cpd01012[e]': 1000, #Cd2+
'EX_cpd11593[e]': 1000, #ala-L-asp-L
'EX_cpd00159[e]': 1000, #L-Lactate
'EX_cpd01262[e]': 1000, #Amylotriose
'EX_cpd01329[e]': 1000, #Maltohexaose
'EX_cpd00242[e]': 1000, #H2CO3
'EX_cpd00013[e]': 1000, #cpd00013[e]
'EX_cpd11640[e]': 1000, #H2
}
model_full.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
hvphoton1[e],EX_PHO1,100,0,0.00%
cpd00007[e],EX_cpd00007[e],325,0,0.00%
cpd00009[e],EX_cpd00009[e],7.601,0,0.00%
cpd00013[e],EX_cpd00013[e],74.52,0,0.00%
cpd00027[e],EX_cpd00027[e],164.6,6,81.61%
cpd00034[e],EX_cpd00034[e],0.0587,0,0.00%
cpd00048[e],EX_cpd00048[e],1.315,0,0.00%
cpd00053[e],EX_cpd00053[e],16.06,5,6.63%
cpd00058[e],EX_cpd00058[e],0.0587,0,0.00%
cpd00060[e],EX_cpd00060[e],5.523,5,2.28%


In [301]:
base_n2_light1_2 = {
    'EX_PHO2': 100.0,
    'EX_PHO1': 100.0,
    'EX_cpd00528[e]': 10.0,
    'EX_cpd00067[e]': 1000.0,
    'EX_cpd00001[e]': 1000.0,
    'EX_cpd00009[e]': 1000.0,
    'EX_cpd00048[e]': 1000.0,
    'EX_cpd00254[e]': 1000.0,
    'EX_cpd10515[e]': 1000.0,
    'EX_cpd00149[e]': 1000.0,
    'EX_cpd00034[e]': 1000.0,
    'EX_cpd00058[e]': 1000.0,
    'EX_cpd20863[e]': 1000.0,
    'EX_cpd00063[e]': 1000.0,
    'EX_cpd00205[e]': 1000.0,
    'EX_cpd00971[e]': 1000.0,
    'EX_cpd11574[e]': 1000.0,
}
default_medium = {
    'EX_PHO2': 100.0,
    'EX_PHO1': 100.0,
    'EX_cpd00067[e]': 1000.0,
    'EX_cpd00001[e]': 1000.0,
    'EX_cpd00009[e]': 1000.0,
    'EX_cpd00048[e]': 1000.0,
    'EX_cpd00254[e]': 1000.0,
    'EX_cpd10515[e]': 1000.0,
    'EX_cpd00149[e]': 1000.0,
    'EX_cpd00034[e]': 1000.0,
    'EX_cpd00058[e]': 1000.0,
    'EX_cpd20863[e]': 1000.0,
    'EX_cpd00063[e]': 1000.0,
    'EX_cpd00205[e]': 1000.0,
    'EX_cpd00971[e]': 1000.0,
    'EX_cpd11574[e]': 1000.0,
    'EX_cpd00528[e]': 10.0,
    'EX_cpd00242[e]': 8.486
}
for k in default_medium:
    cpd = list(model_full.reactions.get_by_id(k).metabolites)[0]
    print(cpd.id, cpd.name)

hvphoton2[e] Photosystem 2 photon
hvphoton1[e] Photosystem 1 photon
cpd00067[e] H+
cpd00001[e] H2O
cpd00009[e] Phosphate
cpd00048[e] Sulfate
cpd00254[e] Mg
cpd10515[e] Fe+2
cpd00149[e] Co2+
cpd00034[e] Zn2+
cpd00058[e] Cu2+
cpd20863[e] Mn2+
cpd00063[e] Ca2+
cpd00205[e] K+
cpd00971[e] Na+
cpd11574[e] Molybdate
cpd00528[e] N2
cpd00242[e] H2CO3


In [323]:
for e in model_full.exchanges:
    cpd = list(e.metabolites)[0]
    print(f"'{e.id}': 1000, #{cpd.name}")

'EX_PHO2': 1000, #Photosystem 2 photon
'EX_PHO1': 1000, #Photosystem 1 photon
'EX_cpd00067[e]': 1000, #H+
'EX_cpd00001[e]': 1000, #H2O
'EX_cpd00007[e]': 1000, #O2
'EX_cpd00009[e]': 1000, #Phosphate
'EX_cpd00048[e]': 1000, #Sulfate
'EX_cpd00209[e]': 1000, #Nitrate
'EX_cpd00254[e]': 1000, #Mg
'EX_cpd10515[e]': 1000, #Fe+2
'EX_cpd00149[e]': 1000, #Co2+
'EX_cpd00034[e]': 1000, #Zn2+
'EX_cpd00058[e]': 1000, #Cu2+
'EX_cpd20863[e]': 1000, #Mn2+
'EX_cpd00063[e]': 1000, #Ca2+
'EX_cpd00205[e]': 1000, #K+
'EX_cpd00971[e]': 1000, #Na+
'EX_cpd11574[e]': 1000, #Molybdate
'EX_cpd00011[e]': 1000, #CO2
'EX_cpd00528[e]': 1000, #N2
'EX_cpd00027[e]': 1000, #D-Glucose
'EX_cpd00053[e]': 1000, #L-Glutamine
'EX_cpd00107[e]': 1000, #L-Leucine
'EX_cpd00118[e]': 1000, #Putrescine
'EX_cpd00129[e]': 1000, #L-Proline
'EX_cpd00156[e]': 1000, #L-Valine
'EX_cpd00179[e]': 1000, #Maltose
'EX_cpd00264[e]': 1000, #Spermidine
'EX_cpd00268[e]': 1000, #H2S2O3
'EX_cpd00322[e]': 1000, #L-Isoleucine
'EX_cpd03725[e]': 1000, #Fe(

In [312]:
import copy
for e in model_full.exchanges:
    if e.id not in base_n2_light1_2:
        cpd = list(e.metabolites)[0]
        m = copy.deepcopy(base_n2_light1_2)
        m.update({'EX_cpd00007[e]': 1000})
        m.update({e.id: 1})
        model_full.medium = m
        v = model_full.slim_optimize()
        print(e.id, cpd.name, v)

EX_cpd00007[e] O2 nan
EX_cpd00209[e] Nitrate nan
EX_cpd00011[e] CO2 0.005403764017101146
EX_cpd00027[e] D-Glucose 0.12362258410257106
EX_cpd00053[e] L-Glutamine 0.09997882008547625
EX_cpd00107[e] L-Leucine nan
EX_cpd00118[e] Putrescine nan
EX_cpd00129[e] L-Proline nan
EX_cpd00156[e] L-Valine nan
EX_cpd00179[e] Maltose nan
EX_cpd00264[e] Spermidine nan
EX_cpd00268[e] H2S2O3 nan
EX_cpd00322[e] L-Isoleucine nan
EX_cpd03725[e] Fe(III)dicitrate nan
EX_cpd00060[e] L-Methionine nan
EX_cpd00104[e] BIOT nan
EX_cpd01012[e] Cd2+ nan
EX_cpd11593[e] ala-L-asp-L nan
EX_cpd00159[e] L-Lactate nan
EX_cpd01262[e] Amylotriose nan
EX_cpd01329[e] Maltohexaose nan
EX_cpd00242[e] H2CO3 0.005403764017096219
EX_cpd00013[e] cpd00013[e] nan
EX_cpd11640[e] H2 nan


In [304]:
import copy
for e in model_full.exchanges:
    if e.id not in base_n2_light1_2:
        cpd = list(e.metabolites)[0]
        m = copy.deepcopy(base_n2_light1_2)
        m.update({e.id: 1})
        model_full.medium = m
        v = model_full.slim_optimize()
        print(e.id, cpd.name, v)

EX_cpd00007[e] O2 nan
EX_cpd00209[e] Nitrate nan
EX_cpd00011[e] CO2 0.005403764017095576
EX_cpd00027[e] D-Glucose 0.12362258410257534
EX_cpd00053[e] L-Glutamine 0.09997882008547528
EX_cpd00107[e] L-Leucine nan
EX_cpd00118[e] Putrescine nan
EX_cpd00129[e] L-Proline nan
EX_cpd00156[e] L-Valine nan
EX_cpd00179[e] Maltose nan
EX_cpd00264[e] Spermidine nan
EX_cpd00268[e] H2S2O3 nan
EX_cpd00322[e] L-Isoleucine nan
EX_cpd03725[e] Fe(III)dicitrate nan
EX_cpd00060[e] L-Methionine nan
EX_cpd00104[e] BIOT nan
EX_cpd01012[e] Cd2+ nan
EX_cpd11593[e] ala-L-asp-L nan
EX_cpd00159[e] L-Lactate nan
EX_cpd01262[e] Amylotriose nan
EX_cpd01329[e] Maltohexaose nan
EX_cpd00242[e] H2CO3 0.005403764017095296
EX_cpd00013[e] cpd00013[e] nan
EX_cpd11640[e] H2 nan


In [300]:
model_full.medium = default_medium
# 61.35 51.03 0.18240098144906766
sol = cobra.flux_analysis.pfba(model_full)
for r in ['ATP_syn_l__hc', 'ATP_syn_p__hc', 'ATP_syn_l__vc', 'ATP_syn_p__vc']:
    print(r, sol.fluxes[r])
    
model_full.summary(sol)

ATP_syn_l__hc 4.075736423833654
ATP_syn_p__hc 0.13238801433210912
ATP_syn_l__vc 11.591623093233354
ATP_syn_p__vc 1.5455133470271156


Metabolite,Reaction,Flux,C-Number,C-Flux
hvphoton1[e],EX_PHO1,61.35,0,0.00%
hvphoton2[e],EX_PHO2,51.03,0,0.00%
cpd00009[e],EX_cpd00009[e],0.07813,0,0.00%
cpd00034[e],EX_cpd00034[e],0.0005935,0,0.00%
cpd00048[e],EX_cpd00048[e],0.07041,0,0.00%
cpd00058[e],EX_cpd00058[e],0.0005935,0,0.00%
cpd00063[e],EX_cpd00063[e],0.0008893,0,0.00%
cpd00067[e],EX_cpd00067[e],19.48,0,0.00%
cpd00149[e],EX_cpd00149[e],0.0006373,0,0.00%
cpd00205[e],EX_cpd00205[e],0.03338,0,0.00%


In [231]:
ws = 87388 # Anabaena sp. ATCC 33047
genome = kbase.get_from_ws('GCF_001597855.1.RAST', ws)

In [233]:
from modelseedpy import RastClient, MSBuilder
rast = RastClient()
rast.annotate_genome(genome)

[{'parameters': ['-a',
   '-g',
   200,
   '-m',
   5,
   '-d',
   '/opt/patric-common/data/kmer_metadata_v2',
   '-u',
   'http://pear.mcs.anl.gov:6100/query'],
  'id': '2D3A7502-0132-11EC-BA47-A5E5BBF382BD',
  'hostname': 'pear',
  'tool_name': 'kmer_search',
  'execution_time': 1629407556.04401},
 {'tool_name': 'KmerAnnotationByFigfam',
  'execution_time': 1629407556.23076,
  'hostname': 'pear',
  'parameters': ['annotate_hypothetical_only=1',
   'dataset_name=Release70',
   'kmer_size=8'],
  'id': '2D56F4FC-0132-11EC-BA47-A5E5BBF382BD'},
 {'id': '2F2423F4-0132-11EC-BBE9-4688BCF382BD',
  'parameters': [],
  'tool_name': 'annotate_proteins_similarity',
  'hostname': 'pear',
  'execute_time': 1629407559.25314}]

In [234]:
template_core = kbase.get_from_ws('CoreBacteriaTest_FIX', 82183)

In [237]:
builder = MSBuilder(genome, template_core)

In [238]:
model_test = builder.build('m', '0', False, False)

In [277]:
model_full.reactions.ATP_syn_p_p

AttributeError: DictList has no attribute or entry ATP_syn_p_p

In [262]:
genome.features.A3776_RS09565.ontology_terms

{'RAST': ['Pyruvate decarboxylase (EC 4.1.1.1)',
  'Indole-3-pyruvate decarboxylase (EC 4.1.1.74)']}

In [335]:
from cobra.core.gene import parse_gpr
expr, variables = parse_gpr("a or (b and c)")

In [ ]:
import pyea

In [342]:
expr.body.values

[<_ast.Name at 0x7fcbc4c76f40>, <_ast.BoolOp at 0x7fcbc4c768e0>]

In [325]:
for r in model_test.reactions:
    i = r.id
    seed_id = i[:-3]
    print(r, r.gene_reaction_rule)
    if f'{seed_id}__vc' in model_full.reactions:
        rxn_other = model_full.reactions.get_by_id(f'{seed_id}__vc')
        print(rxn_other, rxn_other.gene_reaction_rule)
    if f'{seed_id}__hc' in model_full.reactions:
        rxn_other = model_full.reactions.get_by_id(f'{seed_id}__hc')
        print(rxn_other, rxn_other.gene_reaction_rule)
    print()

rxn00549_c0: cpd00001_c0 + cpd00290_c0 --> cpd00009_c0 + cpd00072_c0 (A3776_RS07680) or (A3776_RS19020)
rxn00549__vc: cpd00001[vc] + cpd00290[vc] --> cpd00009[vc] + cpd00072[vc] A3776_RS07680 or A3776_RS19020
rxn00549__hc: cpd00001[hc] + cpd00290[hc] --> cpd00009[hc] + cpd00072[hc] A3776_RS07680 or A3776_RS19020

rxn14427_c0: 4 cpd00067_c0 + cpd00209_c0 + cpd18074_c0 --> cpd00001_c0 + 2 cpd00067_e0 + cpd00075_c0 + cpd18072_c0 (A3776_RS11430) or (A3776_RS11430)

rxn05040_c0: cpd00171_c0 --> cpd00047_c0 + cpd00067_c0 + cpd11225_c0 (A3776_RS22120)
rxn05040__vc: cpd00171[vc] --> cpd00047[vc] + cpd00067[vc] + cpd11225[vc] A3776_RS22120
rxn05040__hc: cpd00171[hc] --> cpd00047[hc] + cpd00067[hc] + cpd11225[hc] A3776_RS22120

rxn00545_c0: cpd00002_c0 + cpd00072_c0 --> cpd00008_c0 + cpd00067_c0 + cpd00290_c0 (A3776_RS16105)

rxn08977_c0: cpd00005_c0 + cpd00067_c0 + cpd15560_c0 --> cpd00006_c0 + cpd15561_c0 (A3776_RS00950)

rxn01333_c0: cpd00102_c0 + cpd00238_c0 <=> cpd00072_c0 + cpd00236_c0 (A3